In [1]:
# from utilities import process_all_files_from_directory
import pandas as pd
import numpy as np
import polars as pl
import os
from decimal import Decimal, getcontext

#### Important functions

In [2]:
def calculate_series(val, discount_rate, start_year, end_year=2050, ftype='discounted'):
    if ftype == 'discounted':
        res = val * ((1 - (1 / (1 + discount_rate)) ** (end_year - start_year + 1)) / (1 - (1 / (1 + discount_rate)))) - val
    elif ftype == 'undiscounted':
        res = val * (end_year - start_year + 1) - val
    else:
        res = None
    return res

In [3]:
def to_decimal(val):
    return Decimal(val) if val is not None else None

def convert_float_columns_to_decimal(df):
    float_columns = [col for col, dtype in zip(df.columns, df.dtypes) if dtype == pl.Float32 or dtype == pl.Float64]
    
    df = df.with_columns([pl.col(col).map_elements(to_decimal) for col in float_columns])
    
    return df


## Set Precision

In [4]:
getcontext().prec = 12

In [5]:
damage_full = pl.read_csv('C:/Users/jkumava/Downloads/PCRM/damage_full_202407161812.csv')
damage_full

entity_id,rcp_scenario,physical_impact_year,expected_revenue_loss_amount_series_undiscounted,expected_asset_damage_loss_amount_series_undiscounted,expected_direct_damage_loss_amount_series_undiscounted,expected_indirect_loss_amount_series_undiscounted,expected_revenue_loss_amount_series_discounted,expected_asset_damage_loss_amount_series_discounted,expected_direct_damage_loss_amount_series_discounted,expected_indirect_loss_amount_series_discounted,expected_total_loss_amount_series_undiscounted,expected_total_loss_amount_series_discounted
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1007896773,"""RCP2.6""",2023,1.6104e7,1.5280e7,3.1384e7,7.4451e6,1.4066e7,1.3346e7,2.7412e7,6.5028e6,3.8829e7,3.3915e7
1007896773,"""RCP2.6""",2024,1.6362e7,1.5464e7,3.1825e7,7.5557e6,1.3356e7,1.2623e7,2.5979e7,6.1677e6,3.9381e7,3.2147e7
1007896773,"""RCP2.6""",2025,1.6620e7,1.5647e7,3.2266e7,7.6664e6,1.2679e7,1.1937e7,2.4616e7,5.8487e6,3.9933e7,3.0465e7
1007896773,"""RCP2.6""",2026,1.6816e7,1.5759e7,3.2575e7,7.8057e6,1.1990e7,1.1236e7,2.3225e7,5.5653e6,4.0380e7,2.8791e7
1007896773,"""RCP2.6""",2027,1.7012e7,1.5870e7,3.2883e7,7.9450e6,1.1336e7,1.0575e7,2.1911e7,5.2941e6,4.0828e7,2.7205e7
…,…,…,…,…,…,…,…,…,…,…,…,…
2013649583,"""RCP2.6""",2046,3.4171e6,2.4549e6,5.8720e6,1.6509e6,629603.070113,452313.040933,1.0819e6,304174.411124,7.5229e6,1.3861e6
2013649583,"""RCP2.6""",2047,3.4898e6,2.5281e6,6.0179e6,1.6944e6,600930.70175,435325.944002,1.0363e6,291773.496587,7.7123e6,1.3280e6
2013649583,"""RCP2.6""",2048,3.5625e6,2.6013e6,6.1638e6,1.7380e6,573315.251666,418625.62602,991940.877687,279692.881061,7.9018e6,1.2716e6


In [6]:
def convert_float_columns_to_decimal(df, scale):
    precision = 36  # Set a precision value high enough for your needs
    float_columns = [col for col, dtype in zip(df.columns, df.dtypes) if dtype in [pl.Float32, pl.Float64]]

    # Cast the float columns to Decimal with specified precision and scale
    df = df.with_columns([pl.col(col).cast(pl.Decimal(scale=scale, precision=precision)).alias(col) for col in float_columns])
    
    return df

In [7]:
damage_full = convert_float_columns_to_decimal(damage_full,12)

In [8]:
# Explicite conversion
# df = df.with_columnss([
#     pl.col("gppe_median_usd").map_elements(to_decimal, return_dtype=pl.Object),
#     pl.col("cash_median_usd").map_elements(to_decimal, return_dtype=pl.Object),
#     pl.col("revenue_median_usd").map_elements(to_decimal, return_dtype=pl.Object),
#     pl.col("gppe_avg_usd").map_elements(to_decimal, return_dtype=pl.Object),
#     pl.col("cash_avg_usd").map_elements(to_decimal, return_dtype=pl.Object),
#     pl.col("revenue_avg_usd").map_elements(to_decimal, return_dtype=pl.Object)
# ])

In [9]:

cash_flow_df = pl.read_csv('C:/Users/jkumava/Downloads/PCRM/company_global_ppe_202407111106.csv')

cash_flow_df = convert_float_columns_to_decimal(cash_flow_df,12)

cash_flow_df = cash_flow_df.with_columns([
    pl.col("cash_avg_usd").map_elements(lambda x: calculate_series(x, 0.07, 2022, 2050, 'discounted'), return_dtype=pl.Float64).alias("cash_flow_series_cumulative_discounted"),
    pl.col("cash_avg_usd").map_elements(lambda x: calculate_series(x, 0.07, 2022, 2050, 'undiscounted'), return_dtype=pl.Float64).alias("cash_flow_series_cumulative_undiscounted")
])

cash_flow_df = cash_flow_df.rename({"sustcompanyid": "entity_id"})
cash_flow_df

entity_id,fiscal_year,gppe_median_usd,cash_median_usd,revenue_median_usd,gppe_avg_usd,cash_avg_usd,revenue_avg_usd,dt_created,dt_updated,fl_deleted,cash_flow_series_cumulative_discounted,cash_flow_series_cumulative_undiscounted
i64,i64,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64
2000140868,2020,2.0877e7,2.7780e6,null,2.0877e7,2.7780e6,null,"""2024-03-22 16:23:13.948""","""2024-03-22 16:23:13.948""",null,3.3717e7,7.7784e7
1008049895,2022,2.2996e9,6.1992e8,3.9619e9,2.2861e9,7.0290e8,4.8270e9,"""2024-01-18 14:16:50.731""","""2024-03-27 13:43:07.434""",null,8.5312e9,1.9681e10
1008177948,2022,1.7273e9,4.362e8,2.3647e9,1.7713e9,4.3828e8,2.6727e9,"""2024-01-18 14:16:50.731""","""2024-03-27 13:43:07.434""",null,5.3195e9,1.2272e10
2001722369,2021,1.6426e7,1.3795e6,null,8.2218e6,-351217.347835,null,"""2024-03-22 16:23:13.948""","""2024-03-22 16:23:13.948""",null,-4.2628e6,-9.8341e6
2004493237,2020,2.1919e8,-1.0678e8,null,2.1919e8,-1.0678e8,null,"""2024-03-22 16:23:13.948""","""2024-03-22 16:23:13.948""",null,-1.2960e9,-2.9898e9
…,…,…,…,…,…,…,…,…,…,…,…,…
2000505558,2022,2.6230e8,1.8468e7,null,2.5956e8,1.7731e7,null,"""2024-01-18 14:16:50.731""","""2024-03-05 11:45:04.755""",null,2.1520e8,4.9647e8
2000181128,2022,241702.163585,-2.2059e6,null,173459.180527,-2.3765e6,null,"""2024-01-18 14:16:50.731""","""2024-03-05 11:45:04.755""",null,-2.8844e7,-6.6541e7
2001710123,2022,2.8048e7,9.0198e6,null,2.8460e7,8.7296e6,null,"""2024-01-18 14:16:50.731""","""2024-03-05 11:45:04.755""",null,1.0595e8,2.4443e8


In [10]:
exposure_agg_df = damage_full.group_by(['entity_id', 'rcp_scenario']).agg(
    expected_indirect_loss_amount_cumulative_undiscounted=pl.col('expected_indirect_loss_amount_series_undiscounted').sum(),
    expected_indirect_loss_amount_cumulative_discounted=pl.col('expected_indirect_loss_amount_series_discounted').sum(),
    expected_revenue_loss_amount_cumulative_undiscounted=pl.col('expected_revenue_loss_amount_series_undiscounted').sum(),
    expected_revenue_loss_amount_cumulative_discounted=pl.col('expected_revenue_loss_amount_series_discounted').sum(),
    expected_asset_damage_loss_amount_cumulative_undiscounted=pl.col('expected_asset_damage_loss_amount_series_undiscounted').sum(),
    expected_asset_damage_loss_amount_cumulative_discounted=pl.col('expected_asset_damage_loss_amount_series_discounted').sum()
)
exposure_agg_df

entity_id,rcp_scenario,expected_indirect_loss_amount_cumulative_undiscounted,expected_indirect_loss_amount_cumulative_discounted,expected_revenue_loss_amount_cumulative_undiscounted,expected_revenue_loss_amount_cumulative_discounted,expected_asset_damage_loss_amount_cumulative_undiscounted,expected_asset_damage_loss_amount_cumulative_discounted
i64,str,f64,f64,f64,f64,f64,f64
1012371436,"""RCP2.6""",4.1773e7,1.5476e7,5.7988e7,2.0599e7,9.6610e7,3.6363e7
1008758403,"""RCP2.6""",3.9306e6,1.5733e6,1.6227e7,6.3514e6,3.0494e7,1.2258e7
2000160058,"""RCP2.6""",2.4831e7,9.1064e6,4.5834e7,1.7241e7,7.6328e7,2.8664e7
1016789142,"""RCP2.6""",1.1195e7,4.3829e6,6.2842e7,2.5353e7,4.2292e7,1.7011e7
1016057001,"""RCP2.6""",9.3272e6,3.5576e6,1.4915e7,5.6449e6,4.4717e7,1.7281e7
…,…,…,…,…,…,…,…
1008755869,"""RCP2.6""",303696.864941,112432.982226,1.1176e6,430943.463304,160469.406049,64040.201139
1013597082,"""RCP2.6""",651637.894915,245753.470403,849792.295741,319050.500293,1.4212e7,5.5002e6
1008761858,"""RCP2.6""",5.0391e6,1.7297e6,9.6672e6,3.3965e6,4.6457e7,1.6374e7


In [11]:
industry_company_df = pl.read_csv('C:/Users/jkumava/Downloads/PCRM/industry_company_202407171650.csv')
industry_company_df = industry_company_df.rename({'entityid': 'entity_id'} )
industry_company_df

entity_id,clientindustryid
i64,i64
2006146113,451050
2003888171,402047
1008159611,403048
1008204897,601049
2007748991,451050
…,…
1008761183,201025
1007910068,352045
2007110142,402047


In [12]:
exposure_datapoints_df = exposure_agg_df\
    .join(cash_flow_df.select(['entity_id', 'cash_flow_series_cumulative_discounted', 'cash_flow_series_cumulative_undiscounted']), on='entity_id')\
    .join(industry_company_df, on='entity_id')
exposure_datapoints_df = exposure_datapoints_df\
    .with_columns((pl.col('expected_revenue_loss_amount_cumulative_undiscounted') +
        pl.col('expected_asset_damage_loss_amount_cumulative_undiscounted')).alias('expected_direct_damage_loss_amount_cumulative_undiscounted')
    )\
    .with_columns((
        pl.col('expected_direct_damage_loss_amount_cumulative_undiscounted') +
        pl.col('expected_indirect_loss_amount_cumulative_undiscounted')).alias('expected_total_loss_amount_cumulative_undiscounted')
)\
.with_columns(
        indirect_loss_ratio_undiscounted = \
        pl.col('expected_indirect_loss_amount_cumulative_undiscounted') /
        pl.when(pl.col('cash_flow_series_cumulative_undiscounted') == 0).then(pl.lit(None))\
          .otherwise(pl.col('cash_flow_series_cumulative_undiscounted')),
direct_loss_ratio_undiscounted=
        (pl.col('expected_direct_damage_loss_amount_cumulative_undiscounted') /
         pl.when(pl.col('cash_flow_series_cumulative_undiscounted') == 0).then(pl.lit(None))\
           .otherwise(pl.col('cash_flow_series_cumulative_undiscounted'))))\
.with_columns(
total_loss_ratio_undiscounted = (pl.col('indirect_loss_ratio_undiscounted') +
        pl.col('direct_loss_ratio_undiscounted')),
    expected_direct_damage_loss_amount_cumulative_discounted=(
        pl.col('expected_revenue_loss_amount_cumulative_discounted') +
        pl.col('expected_asset_damage_loss_amount_cumulative_discounted')))\
.with_columns(
    expected_total_loss_amount_cumulative_discounted = (
pl.col('expected_direct_damage_loss_amount_cumulative_discounted') +
        pl.col('expected_indirect_loss_amount_cumulative_discounted')
    ),
    indirect_loss_ratio_discounted = pl.col('expected_indirect_loss_amount_cumulative_discounted') /
        (pl.when(pl.col('cash_flow_series_cumulative_discounted') == 0).then(pl.lit(None))\
          .otherwise(pl.col('cash_flow_series_cumulative_discounted'))
    ),
direct_loss_ratio_discounted = pl.col('expected_direct_damage_loss_amount_cumulative_discounted') /
        (pl.when(pl.col('cash_flow_series_cumulative_discounted') == 0).then( pl.lit(None))\
          .otherwise(pl.col('cash_flow_series_cumulative_discounted')))
)\
.with_columns(
total_loss_ratio_discounted = (pl.col('indirect_loss_ratio_discounted') +
        pl.col('direct_loss_ratio_discounted')))

exposure_datapoints_df

entity_id,rcp_scenario,expected_indirect_loss_amount_cumulative_undiscounted,expected_indirect_loss_amount_cumulative_discounted,expected_revenue_loss_amount_cumulative_undiscounted,expected_revenue_loss_amount_cumulative_discounted,expected_asset_damage_loss_amount_cumulative_undiscounted,expected_asset_damage_loss_amount_cumulative_discounted,cash_flow_series_cumulative_discounted,cash_flow_series_cumulative_undiscounted,clientindustryid,expected_direct_damage_loss_amount_cumulative_undiscounted,expected_total_loss_amount_cumulative_undiscounted,indirect_loss_ratio_undiscounted,direct_loss_ratio_undiscounted,total_loss_ratio_undiscounted,expected_direct_damage_loss_amount_cumulative_discounted,expected_total_loss_amount_cumulative_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,total_loss_ratio_discounted
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2006146113,"""RCP2.6""",153897.616615,61166.587834,6647.278731,2679.201901,30172.599244,12223.237442,5.9855e8,1.3808e9,451050,36819.877976,190717.494591,0.000111,0.000027,0.000138,14902.439343,76069.027177,0.000102,0.000025,0.000127
2007748991,"""RCP2.6""",4.0713e6,1.5137e6,1.8410e7,7.0228e6,4.6600e6,1.8147e6,3.7711e9,8.6999e9,451050,2.3070e7,2.7141e7,0.000468,0.002652,0.00312,8.8375e6,1.0351e7,0.000401,0.002343,0.002745
2000219494,"""RCP2.6""",1.0644e7,4.0284e6,5.6265e7,2.1638e7,304985.747078,122007.876358,2.4779e9,5.7165e9,254039,5.6570e7,6.7214e7,0.001862,0.009896,0.011758,2.1760e7,2.5788e7,0.001626,0.008781,0.010407
1008243682,"""RCP2.6""",1.2927e8,4.8598e7,4.0181e8,1.5205e8,1.4896e8,5.8016e7,1.7051e10,3.9336e10,255040,5.5077e8,6.8004e8,0.003286,0.014002,0.017288,2.1007e8,2.5867e8,0.00285,0.01232,0.01517
1008754510,"""RCP2.6""",2.7884e7,1.0554e7,8.5092e7,3.2128e7,5.5351e8,2.1826e8,2.3578e10,5.4393e10,501053,6.3860e8,6.6649e8,0.000513,0.01174,0.012253,2.5039e8,2.6094e8,0.000448,0.01062,0.011067
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2006980355,"""RCP2.6""",2.4281e8,9.2077e7,6.2070e8,2.3603e8,1.3181e8,4.9950e7,1.9211e10,4.4318e10,201028,7.5251e8,9.9532e8,0.005479,0.01698,0.022459,2.8598e8,3.7806e8,0.004793,0.014887,0.01968
1030215664,"""RCP2.6""",1.0065e7,3.8230e6,5.3272e7,2.0544e7,1.1577e7,4.5569e6,4.2687e9,9.8477e9,252036,6.4848e7,7.4913e7,0.001022,0.006585,0.007607,2.5101e7,2.8924e7,0.000896,0.00588,0.006776
1008759007,"""RCP2.6""",2.4864e7,8.9679e6,3.6339e7,1.3585e7,2.8492e7,1.0498e7,9.0695e8,2.0923e9,151015,6.4831e7,8.9695e7,0.011883,0.030985,0.042869,2.4083e7,3.3051e7,0.009888,0.026554,0.036442


In [13]:
filtered_df = exposure_datapoints_df.filter(pl.col('total_loss_ratio_undiscounted').is_not_null())

max_df = filtered_df.group_by('rcp_scenario').agg(
    max_total_loss_ratio_discounted = pl.col('total_loss_ratio_discounted').max(),
    max_indirect_loss_ratio_discounted = pl.col('indirect_loss_ratio_discounted').max(),
    max_direct_loss_ratio_discounted = pl.col('direct_loss_ratio_discounted').max()
)

max_values = filtered_df.join(max_df, on='rcp_scenario', how='inner')

# Selecting required columns
max_values = max_values[
    'entity_id',
    'rcp_scenario',
    'clientindustryid',
    'total_loss_ratio_discounted',
    'indirect_loss_ratio_discounted',
    'direct_loss_ratio_discounted',
    'max_total_loss_ratio_discounted',
    'max_indirect_loss_ratio_discounted',
    'max_direct_loss_ratio_discounted'
]
max_values

entity_id,rcp_scenario,clientindustryid,total_loss_ratio_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,max_total_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_direct_loss_ratio_discounted
i64,str,i64,f64,f64,f64,f64,f64,f64
2006146113,"""RCP2.6""",451050,0.000127,0.000102,0.000025,100.572698,6.132625,94.440072
2007748991,"""RCP2.6""",451050,0.002745,0.000401,0.002343,100.572698,6.132625,94.440072
2000219494,"""RCP2.6""",254039,0.010407,0.001626,0.008781,100.572698,6.132625,94.440072
1008243682,"""RCP2.6""",255040,0.01517,0.00285,0.01232,100.572698,6.132625,94.440072
1008754510,"""RCP2.6""",501053,0.011067,0.000448,0.01062,100.572698,6.132625,94.440072
…,…,…,…,…,…,…,…,…
2006980355,"""RCP2.6""",201028,0.01968,0.004793,0.014887,100.572698,6.132625,94.440072
1030215664,"""RCP2.6""",252036,0.006776,0.000896,0.00588,100.572698,6.132625,94.440072
1008759007,"""RCP2.6""",151015,0.036442,0.009888,0.026554,100.572698,6.132625,94.440072


In [14]:
max_values = max_values.with_columns([
    pl.when(pl.col('total_loss_ratio_discounted') < 0).then\
            (pl.col('max_total_loss_ratio_discounted') + abs(pl.col('total_loss_ratio_discounted')))
    .otherwise(pl.col('total_loss_ratio_discounted')).alias('adjusted_total_loss_ratio_discounted'),
    
    pl.when(pl.col('direct_loss_ratio_discounted') < 0).then\
            (pl.col('max_direct_loss_ratio_discounted') + abs(pl.col('direct_loss_ratio_discounted')))
    .otherwise(pl.col('direct_loss_ratio_discounted')).alias('adjusted_direct_loss_ratio_discounted'),
    
    pl.when(pl.col('indirect_loss_ratio_discounted') < 0).then\
            (pl.col('max_indirect_loss_ratio_discounted') + abs(pl.col('indirect_loss_ratio_discounted')))
    .otherwise(pl.col('indirect_loss_ratio_discounted')).alias('adjusted_indirect_loss_ratio_discounted')
])
max_values

entity_id,rcp_scenario,clientindustryid,total_loss_ratio_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,max_total_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_direct_loss_ratio_discounted,adjusted_total_loss_ratio_discounted,adjusted_direct_loss_ratio_discounted,adjusted_indirect_loss_ratio_discounted
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2006146113,"""RCP2.6""",451050,0.000127,0.000102,0.000025,100.572698,6.132625,94.440072,0.000127,0.000025,0.000102
2007748991,"""RCP2.6""",451050,0.002745,0.000401,0.002343,100.572698,6.132625,94.440072,0.002745,0.002343,0.000401
2000219494,"""RCP2.6""",254039,0.010407,0.001626,0.008781,100.572698,6.132625,94.440072,0.010407,0.008781,0.001626
1008243682,"""RCP2.6""",255040,0.01517,0.00285,0.01232,100.572698,6.132625,94.440072,0.01517,0.01232,0.00285
1008754510,"""RCP2.6""",501053,0.011067,0.000448,0.01062,100.572698,6.132625,94.440072,0.011067,0.01062,0.000448
…,…,…,…,…,…,…,…,…,…,…,…
2006980355,"""RCP2.6""",201028,0.01968,0.004793,0.014887,100.572698,6.132625,94.440072,0.01968,0.014887,0.004793
1030215664,"""RCP2.6""",252036,0.006776,0.000896,0.00588,100.572698,6.132625,94.440072,0.006776,0.00588,0.000896
1008759007,"""RCP2.6""",151015,0.036442,0.009888,0.026554,100.572698,6.132625,94.440072,0.036442,0.026554,0.009888


In [15]:
exposure_datapoints_df.columns

['entity_id',
 'rcp_scenario',
 'expected_indirect_loss_amount_cumulative_undiscounted',
 'expected_indirect_loss_amount_cumulative_discounted',
 'expected_revenue_loss_amount_cumulative_undiscounted',
 'expected_revenue_loss_amount_cumulative_discounted',
 'expected_asset_damage_loss_amount_cumulative_undiscounted',
 'expected_asset_damage_loss_amount_cumulative_discounted',
 'cash_flow_series_cumulative_discounted',
 'cash_flow_series_cumulative_undiscounted',
 'clientindustryid',
 'expected_direct_damage_loss_amount_cumulative_undiscounted',
 'expected_total_loss_amount_cumulative_undiscounted',
 'indirect_loss_ratio_undiscounted',
 'direct_loss_ratio_undiscounted',
 'total_loss_ratio_undiscounted',
 'expected_direct_damage_loss_amount_cumulative_discounted',
 'expected_total_loss_amount_cumulative_discounted',
 'indirect_loss_ratio_discounted',
 'direct_loss_ratio_discounted',
 'total_loss_ratio_discounted']

In [16]:
max_values.columns

['entity_id',
 'rcp_scenario',
 'clientindustryid',
 'total_loss_ratio_discounted',
 'indirect_loss_ratio_discounted',
 'direct_loss_ratio_discounted',
 'max_total_loss_ratio_discounted',
 'max_indirect_loss_ratio_discounted',
 'max_direct_loss_ratio_discounted',
 'adjusted_total_loss_ratio_discounted',
 'adjusted_direct_loss_ratio_discounted',
 'adjusted_indirect_loss_ratio_discounted']

In [17]:
tier_conditions =  pl.read_csv('tier_conditions.csv')

def categorize_level(level, tier_conditions, tier_type):
    if level is None:
        return None
    tier_country = tier_conditions.filter(pl.col('tier_type') == 'exposure_percentile').drop('tier_type').to_dicts()

    for row in tier_country:
        if (row['lowerbound'] < level <= row['upperbound']):
            return row['tier']
    return None


def calculate_percentiles_and_ranks(df, value_column, group_by_columns):
    df = df.clone()
    df = df.with_columns([
        (pl.col(value_column).rank(method='dense').over(group_by_columns).cast(pl.Float64) / (df[value_column].len() - 1) * 100).alias('percentile'),
        pl.col(value_column).rank(method='min').over(group_by_columns).cast(pl.Int32).alias('rank')
    ], maintain_order=True)
    return df
    
def calculate_percentiles(df, value_column, group_by_columns):
    df = df.clone()
    df = df.with_columns([
        (pl.col(value_column).rank(method='dense').over(group_by_columns).cast(pl.Float64) / (df[value_column].len() - 1) * 100).alias('percentile'),
    ], maintain_order=True)
    return df

def calculate_ranks(df, value_column, group_by_columns):
    df = df.clone()
    df = df.with_columns([
        pl.col(value_column).rank(method='min').over(group_by_columns).cast(pl.Int32).alias('rank')
    ], maintain_order=True)
    return df
    

In [18]:
# max_values.with_columns([
#         pl.col('adjusted_total_loss_ratio_discounted').rank(method='min').over(['rcp_scenario', 'clientindustryid']).cast(pl.Int32).alias('rank')
#     ], maintain_order=True)

# calculate_percentiles(max_values, 'adjusted_total_loss_ratio_discounted', ['rcp_scenario', 'clientindustryid'])

In [19]:
max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_total_loss_ratio_discounted', ['rcp_scenario'])
max_values = max_values.with_columns(overall_percentile_total_loss_ratio=pl.col('percentile').round(),
                                     total_loss_ratio_rank_overall= pl.col('rank')
)
max_values

entity_id,rcp_scenario,clientindustryid,total_loss_ratio_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,max_total_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_direct_loss_ratio_discounted,adjusted_total_loss_ratio_discounted,adjusted_direct_loss_ratio_discounted,adjusted_indirect_loss_ratio_discounted,percentile,rank,maintain_order,overall_percentile_total_loss_ratio,total_loss_ratio_rank_overall
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,bool,f64,i32
2006146113,"""RCP2.6""",451050,0.000127,0.000102,0.000025,100.572698,6.132625,94.440072,0.000127,0.000025,0.000102,0.558544,27,true,1.0,27
2007748991,"""RCP2.6""",451050,0.002745,0.000401,0.002343,100.572698,6.132625,94.440072,0.002745,0.002343,0.000401,9.453868,457,true,9.0,457
2000219494,"""RCP2.6""",254039,0.010407,0.001626,0.008781,100.572698,6.132625,94.440072,0.010407,0.008781,0.001626,36.326024,1756,true,36.0,1756
1008243682,"""RCP2.6""",255040,0.01517,0.00285,0.01232,100.572698,6.132625,94.440072,0.01517,0.01232,0.00285,48.324369,2336,true,48.0,2336
1008754510,"""RCP2.6""",501053,0.011067,0.000448,0.01062,100.572698,6.132625,94.440072,0.011067,0.01062,0.000448,38.249897,1849,true,38.0,1849
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2006980355,"""RCP2.6""",201028,0.01968,0.004793,0.014887,100.572698,6.132625,94.440072,0.01968,0.014887,0.004793,56.681837,2740,true,57.0,2740
1030215664,"""RCP2.6""",252036,0.006776,0.000896,0.00588,100.572698,6.132625,94.440072,0.006776,0.00588,0.000896,24.741415,1196,true,25.0,1196
1008759007,"""RCP2.6""",151015,0.036442,0.009888,0.026554,100.572698,6.132625,94.440072,0.036442,0.026554,0.009888,73.293339,3543,true,73.0,3543


In [20]:
max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_total_loss_ratio_discounted', ['rcp_scenario'])
max_values = max_values.with_columns(overall_percentile_total_loss_ratio=pl.col('percentile').round(),
                                     total_loss_ratio_rank_overall= pl.col('rank') )

max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_total_loss_ratio_discounted', ['rcp_scenario', 'clientindustryid'])
max_values = max_values.with_columns(industry_percentile_total_loss_ratio=pl.col('percentile').round(),
                                     total_loss_ratio_rank_industry= pl.col('rank') )


max_values = calculate_percentiles(max_values, 'adjusted_direct_loss_ratio_discounted', ['rcp_scenario'])
max_values = max_values.with_columns(overall_percentile_direct_loss_ratio=pl.col('percentile').round() )

max_values = calculate_percentiles(max_values, 'adjusted_direct_loss_ratio_discounted', ['rcp_scenario', 'clientindustryid'])
max_values = max_values.with_columns(industry_percentile_direct_loss_ratio=pl.col('percentile').round() )

max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_indirect_loss_ratio_discounted', ['rcp_scenario'])
max_values = max_values.with_columns(overall_percentile_indirect_loss_ratio= pl.col('percentile').round()
)

max_values = calculate_percentiles(max_values, 'adjusted_indirect_loss_ratio_discounted', ['rcp_scenario', 'clientindustryid'])
max_values = max_values.with_columns(industry_percentile_indirect_loss_ratio = pl.col('percentile').round()
)

max_values = max_values.drop(['percentile', 'rank'])

max_values = max_values.with_columns(
    percent_total_loss_from_direct =
    (pl.col('direct_loss_ratio_discounted') / (pl.col('indirect_loss_ratio_discounted') + pl.col('direct_loss_ratio_discounted'))) * 100
)

max_values = max_values.with_columns(
    percent_total_loss_from_indirect=
    (pl.col('indirect_loss_ratio_discounted') / (pl.col('indirect_loss_ratio_discounted') + pl.col('direct_loss_ratio_discounted'))) * 100
)
max_values

entity_id,rcp_scenario,clientindustryid,total_loss_ratio_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,max_total_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_direct_loss_ratio_discounted,adjusted_total_loss_ratio_discounted,adjusted_direct_loss_ratio_discounted,adjusted_indirect_loss_ratio_discounted,maintain_order,overall_percentile_total_loss_ratio,total_loss_ratio_rank_overall,industry_percentile_total_loss_ratio,total_loss_ratio_rank_industry,overall_percentile_direct_loss_ratio,industry_percentile_direct_loss_ratio,overall_percentile_indirect_loss_ratio,industry_percentile_indirect_loss_ratio,percent_total_loss_from_direct,percent_total_loss_from_indirect
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,f64,i32,f64,i32,f64,f64,f64,f64,f64,f64
2006146113,"""RCP2.6""",451050,0.000127,0.000102,0.000025,100.572698,6.132625,94.440072,0.000127,0.000025,0.000102,true,1.0,27,0.0,2,0.0,0.0,3.0,0.0,19.59068,80.40932
2007748991,"""RCP2.6""",451050,0.002745,0.000401,0.002343,100.572698,6.132625,94.440072,0.002745,0.002343,0.000401,true,9.0,457,2.0,97,11.0,2.0,9.0,1.0,85.376536,14.623464
2000219494,"""RCP2.6""",254039,0.010407,0.001626,0.008781,100.572698,6.132625,94.440072,0.010407,0.008781,0.001626,true,36.0,1756,1.0,58,38.0,1.0,39.0,1.0,84.378802,15.621198
1008243682,"""RCP2.6""",255040,0.01517,0.00285,0.01232,100.572698,6.132625,94.440072,0.01517,0.01232,0.00285,true,48.0,2336,2.0,88,47.0,2.0,58.0,2.0,81.212395,18.787605
1008754510,"""RCP2.6""",501053,0.011067,0.000448,0.01062,100.572698,6.132625,94.440072,0.011067,0.01062,0.000448,true,38.0,1849,1.0,49,43.0,1.0,10.0,0.0,95.955247,4.044753
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2006980355,"""RCP2.6""",201028,0.01968,0.004793,0.014887,100.572698,6.132625,94.440072,0.01968,0.014887,0.004793,true,57.0,2740,1.0,50,53.0,1.0,72.0,1.0,75.644888,24.355112
1030215664,"""RCP2.6""",252036,0.006776,0.000896,0.00588,100.572698,6.132625,94.440072,0.006776,0.00588,0.000896,true,25.0,1196,0.0,18,27.0,0.0,23.0,0.0,86.782399,13.217601
1008759007,"""RCP2.6""",151015,0.036442,0.009888,0.026554,100.572698,6.132625,94.440072,0.036442,0.026554,0.009888,true,73.0,3543,4.0,176,70.0,3.0,84.0,4.0,72.866728,27.133272


In [21]:
max_values.filter(pl.col('entity_id')==1007896773)

entity_id,rcp_scenario,clientindustryid,total_loss_ratio_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,max_total_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_direct_loss_ratio_discounted,adjusted_total_loss_ratio_discounted,adjusted_direct_loss_ratio_discounted,adjusted_indirect_loss_ratio_discounted,maintain_order,overall_percentile_total_loss_ratio,total_loss_ratio_rank_overall,industry_percentile_total_loss_ratio,total_loss_ratio_rank_industry,overall_percentile_direct_loss_ratio,industry_percentile_direct_loss_ratio,overall_percentile_indirect_loss_ratio,industry_percentile_indirect_loss_ratio,percent_total_loss_from_direct,percent_total_loss_from_indirect
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,f64,i32,f64,i32,f64,f64,f64,f64,f64,f64
1007896773,"""RCP2.6""",201025,0.018514,0.003768,0.014746,100.572698,6.132625,94.440072,0.018514,0.014746,0.003768,true,55.0,2653,1.0,43,53.0,1.0,67.0,1.0,79.646154,20.353846


In [22]:
# max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_total_loss_ratio_discounted', ['rcp_scenario'])
# max_values['overall_percentile_total_loss_ratio'] = max_values['percentile'].round(0)
# max_values['total_loss_ratio_rank_overall'] = max_values['rank']

# max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_total_loss_ratio_discounted', ['rcp_scenario', 'clientindustryid'])
# max_values['industry_percentile_total_loss_ratio'] = max_values['percentile'].round(0)
# max_values['total_loss_ratio_rank_industry'] = max_values['rank']

# max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_direct_loss_ratio_discounted', ['rcp_scenario'])
# max_values['overall_percentile_direct_loss_ratio'] = max_values['percentile'].round(0)

# max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_direct_loss_ratio_discounted', ['rcp_scenario', 'clientindustryid'])
# max_values['industry_percentile_direct_loss_ratio'] = max_values['percentile'].round(0)

# max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_indirect_loss_ratio_discounted', ['rcp_scenario'])
# max_values['overall_percentile_indirect_loss_ratio'] = max_values['percentile'].round(0).round(0)

# max_values = calculate_percentiles_and_ranks(max_values, 'adjusted_indirect_loss_ratio_discounted', ['rcp_scenario', 'clientindustryid'])
# max_values['industry_percentile_indirect_loss_ratio'] = max_values['percentile'].round(0)

# max_values = max_values.drop(['percentile','rank'],axis=1)

# max_values['percent_total_loss_from_direct'] = (max_values['direct_loss_ratio_discounted'] / 
#                                                (max_values['indirect_loss_ratio_discounted'] + max_values['direct_loss_ratio_discounted'])) * 100

# max_values['percent_total_loss_from_indirect'] = (max_values['indirect_loss_ratio_discounted'] / 
#                                                  (max_values['indirect_loss_ratio_discounted'] + max_values['direct_loss_ratio_discounted'])) * 100

max_values['overall_percentile_direct_loss_ratio']

overall_percentile_direct_loss_ratio
f64
0.0
11.0
38.0
47.0
43.0
…
53.0
27.0
70.0


In [23]:
# max_values.with_columns(pl.col('direct_loss_ratio_discounted').is_null())


# level = max_values['overall_percentile_direct_loss_ratio'][5]
# if level is None:
#     print('Null')
#         # return None
# tier_country = tier_conditions.filter(pl.col('tier_type') == 'exposure_percentile').drop('tier_type').to_dicts()

# for row in tier_country:
#     if (row['lowerbound'] < level <= row['upperbound']):
#         print(row['tier'])
max_values.with_columns(
        pl.when(pl.col('direct_loss_ratio_discounted') >= 0)
            .then(pl.col('overall_percentile_direct_loss_ratio').map_elements(lambda x: categorize_level(x, tier_conditions, 'exposure_percentile'),
                                                                              return_dtype=pl.String))\
            .otherwise(pl.lit(categorize_level(-2, tier_conditions, 'exposure_percentile')))
            .alias('overall_category_direct_loss_ratio'))

entity_id,rcp_scenario,clientindustryid,total_loss_ratio_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,max_total_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_direct_loss_ratio_discounted,adjusted_total_loss_ratio_discounted,adjusted_direct_loss_ratio_discounted,adjusted_indirect_loss_ratio_discounted,maintain_order,overall_percentile_total_loss_ratio,total_loss_ratio_rank_overall,industry_percentile_total_loss_ratio,total_loss_ratio_rank_industry,overall_percentile_direct_loss_ratio,industry_percentile_direct_loss_ratio,overall_percentile_indirect_loss_ratio,industry_percentile_indirect_loss_ratio,percent_total_loss_from_direct,percent_total_loss_from_indirect,overall_category_direct_loss_ratio
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,f64,i32,f64,i32,f64,f64,f64,f64,f64,f64,str
2006146113,"""RCP2.6""",451050,0.000127,0.000102,0.000025,100.572698,6.132625,94.440072,0.000127,0.000025,0.000102,true,1.0,27,0.0,2,0.0,0.0,3.0,0.0,19.59068,80.40932,"""Negligible"""
2007748991,"""RCP2.6""",451050,0.002745,0.000401,0.002343,100.572698,6.132625,94.440072,0.002745,0.002343,0.000401,true,9.0,457,2.0,97,11.0,2.0,9.0,1.0,85.376536,14.623464,"""Medium"""
2000219494,"""RCP2.6""",254039,0.010407,0.001626,0.008781,100.572698,6.132625,94.440072,0.010407,0.008781,0.001626,true,36.0,1756,1.0,58,38.0,1.0,39.0,1.0,84.378802,15.621198,"""High"""
1008243682,"""RCP2.6""",255040,0.01517,0.00285,0.01232,100.572698,6.132625,94.440072,0.01517,0.01232,0.00285,true,48.0,2336,2.0,88,47.0,2.0,58.0,2.0,81.212395,18.787605,"""High"""
1008754510,"""RCP2.6""",501053,0.011067,0.000448,0.01062,100.572698,6.132625,94.440072,0.011067,0.01062,0.000448,true,38.0,1849,1.0,49,43.0,1.0,10.0,0.0,95.955247,4.044753,"""High"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2006980355,"""RCP2.6""",201028,0.01968,0.004793,0.014887,100.572698,6.132625,94.440072,0.01968,0.014887,0.004793,true,57.0,2740,1.0,50,53.0,1.0,72.0,1.0,75.644888,24.355112,"""High"""
1030215664,"""RCP2.6""",252036,0.006776,0.000896,0.00588,100.572698,6.132625,94.440072,0.006776,0.00588,0.000896,true,25.0,1196,0.0,18,27.0,0.0,23.0,0.0,86.782399,13.217601,"""High"""
1008759007,"""RCP2.6""",151015,0.036442,0.009888,0.026554,100.572698,6.132625,94.440072,0.036442,0.026554,0.009888,true,73.0,3543,4.0,176,70.0,3.0,84.0,4.0,72.866728,27.133272,"""High"""


In [24]:
def categorize_level_polars(df):
    return df.with_columns([
        pl.when(pl.col('direct_loss_ratio_discounted') >= 0)
            .then(pl.col('overall_percentile_direct_loss_ratio').map_elements(lambda x: categorize_level(x, tier_conditions, 'exposure_percentile'),
                                                                              return_dtype=pl.Utf8))
            .otherwise(pl.lit(categorize_level(-2, tier_conditions, 'exposure_percentile')))
            .alias('overall_category_direct_loss_ratio'),

        pl.when(pl.col('direct_loss_ratio_discounted') >= 0)
            .then(pl.col('industry_percentile_direct_loss_ratio').map_elements(lambda x: categorize_level(x, tier_conditions, 'exposure_percentile'),return_dtype=pl.Utf8))
            .otherwise(pl.lit(categorize_level(-2, tier_conditions, 'exposure_percentile')))
            .alias('industry_category_direct_loss_ratio'),

        pl.when(pl.col('indirect_loss_ratio_discounted') >= 0)
            .then(pl.col('overall_percentile_indirect_loss_ratio').map_elements(lambda x: categorize_level(x, tier_conditions, 'exposure_percentile'),return_dtype=pl.Utf8))
            .otherwise(pl.lit(categorize_level(-2, tier_conditions, 'exposure_percentile')))
            .alias('overall_category_indirect_loss_ratio'),

        pl.when(pl.col('indirect_loss_ratio_discounted') >= 0)
            .then(pl.col('industry_percentile_indirect_loss_ratio').map_elements(lambda x: categorize_level(x, tier_conditions, 'exposure_percentile'),return_dtype=pl.Utf8))
            .otherwise(pl.lit(categorize_level(-2, tier_conditions, 'exposure_percentile')))
            .alias('industry_category_indirect_loss_ratio'),

        pl.when(pl.col('total_loss_ratio_discounted') >= 0)
            .then((pl.col('indirect_loss_ratio_discounted').round(2) + pl.col('direct_loss_ratio_discounted').round(2))
                .map_elements(lambda x: categorize_level(x, tier_conditions, 'exposure'),return_dtype=pl.Utf8))
            .otherwise(pl.lit(categorize_level(-2, tier_conditions, 'exposure')))
            .alias('total_loss_ratio_category')
    ])

max_values = categorize_level_polars(max_values)
max_values

entity_id,rcp_scenario,clientindustryid,total_loss_ratio_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,max_total_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_direct_loss_ratio_discounted,adjusted_total_loss_ratio_discounted,adjusted_direct_loss_ratio_discounted,adjusted_indirect_loss_ratio_discounted,maintain_order,overall_percentile_total_loss_ratio,total_loss_ratio_rank_overall,industry_percentile_total_loss_ratio,total_loss_ratio_rank_industry,overall_percentile_direct_loss_ratio,industry_percentile_direct_loss_ratio,overall_percentile_indirect_loss_ratio,industry_percentile_indirect_loss_ratio,percent_total_loss_from_direct,percent_total_loss_from_indirect,overall_category_direct_loss_ratio,industry_category_direct_loss_ratio,overall_category_indirect_loss_ratio,industry_category_indirect_loss_ratio,total_loss_ratio_category
i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,f64,i32,f64,i32,f64,f64,f64,f64,f64,f64,str,str,str,str,str
2006146113,"""RCP2.6""",451050,0.000127,0.000102,0.000025,100.572698,6.132625,94.440072,0.000127,0.000025,0.000102,true,1.0,27,0.0,2,0.0,0.0,3.0,0.0,19.59068,80.40932,"""Negligible""","""Negligible""","""Low""","""Negligible""","""Negligible"""
2007748991,"""RCP2.6""",451050,0.002745,0.000401,0.002343,100.572698,6.132625,94.440072,0.002745,0.002343,0.000401,true,9.0,457,2.0,97,11.0,2.0,9.0,1.0,85.376536,14.623464,"""Medium""","""Low""","""Medium""","""Negligible""","""Negligible"""
2000219494,"""RCP2.6""",254039,0.010407,0.001626,0.008781,100.572698,6.132625,94.440072,0.010407,0.008781,0.001626,true,36.0,1756,1.0,58,38.0,1.0,39.0,1.0,84.378802,15.621198,"""High""","""Negligible""","""High""","""Negligible""","""Negligible"""
1008243682,"""RCP2.6""",255040,0.01517,0.00285,0.01232,100.572698,6.132625,94.440072,0.01517,0.01232,0.00285,true,48.0,2336,2.0,88,47.0,2.0,58.0,2.0,81.212395,18.787605,"""High""","""Low""","""High""","""Low""","""Negligible"""
1008754510,"""RCP2.6""",501053,0.011067,0.000448,0.01062,100.572698,6.132625,94.440072,0.011067,0.01062,0.000448,true,38.0,1849,1.0,49,43.0,1.0,10.0,0.0,95.955247,4.044753,"""High""","""Negligible""","""Medium""","""Negligible""","""Negligible"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2006980355,"""RCP2.6""",201028,0.01968,0.004793,0.014887,100.572698,6.132625,94.440072,0.01968,0.014887,0.004793,true,57.0,2740,1.0,50,53.0,1.0,72.0,1.0,75.644888,24.355112,"""High""","""Negligible""","""High""","""Negligible""","""Negligible"""
1030215664,"""RCP2.6""",252036,0.006776,0.000896,0.00588,100.572698,6.132625,94.440072,0.006776,0.00588,0.000896,true,25.0,1196,0.0,18,27.0,0.0,23.0,0.0,86.782399,13.217601,"""High""","""Negligible""","""Medium""","""Negligible""","""Negligible"""
1008759007,"""RCP2.6""",151015,0.036442,0.009888,0.026554,100.572698,6.132625,94.440072,0.036442,0.026554,0.009888,true,73.0,3543,4.0,176,70.0,3.0,84.0,4.0,72.866728,27.133272,"""High""","""Low""","""Severe""","""Low""","""Negligible"""


In [25]:
# max_values['overall_category_direct_loss_ratio'] = max_values.apply(
#     lambda row: categorize_level(row['overall_percentile_direct_loss_ratio'], tier_conditions, 'exposure_percentile') 
#                 if row['direct_loss_ratio_discounted'] >= 0 else categorize_level(-2, tier_conditions, 'exposure_percentile'), axis=1)

# max_values['industry_category_direct_loss_ratio'] = max_values.apply(
#     lambda row: categorize_level(row['industry_percentile_direct_loss_ratio'], tier_conditions, 'exposure_percentile') 
#                 if row['direct_loss_ratio_discounted'] >= 0 else categorize_level(-2, tier_conditions, 'exposure_percentile'), axis=1)

# max_values['overall_category_indirect_loss_ratio'] = max_values.apply(
#     lambda row: categorize_level(row['overall_percentile_indirect_loss_ratio'], tier_conditions, 'exposure_percentile') 
#                 if row['indirect_loss_ratio_discounted'] >= 0 else categorize_level(-2, tier_conditions, 'exposure_percentile'), axis=1)

# max_values['industry_category_indirect_loss_ratio'] = max_values.apply(
#     lambda row: categorize_level(row['industry_percentile_indirect_loss_ratio'], tier_conditions, 'exposure_percentile') 
#                 if row['indirect_loss_ratio_discounted'] >= 0 else categorize_level(-2, tier_conditions, 'exposure_percentile'), axis=1)

# max_values['total_loss_ratio_category'] = max_values.apply(
#     lambda row: categorize_level(round(row['indirect_loss_ratio_discounted'], 2) + round(row['direct_loss_ratio_discounted'], 2), tier_conditions, 'exposure') 
#                 if row['total_loss_ratio_discounted'] >= 0 else categorize_level(-2, tier_conditions, 'exposure'), axis=1)


In [26]:
set(exposure_datapoints_df.columns) - set(max_values.columns)

{'cash_flow_series_cumulative_discounted',
 'cash_flow_series_cumulative_undiscounted',
 'direct_loss_ratio_undiscounted',
 'expected_asset_damage_loss_amount_cumulative_discounted',
 'expected_asset_damage_loss_amount_cumulative_undiscounted',
 'expected_direct_damage_loss_amount_cumulative_discounted',
 'expected_direct_damage_loss_amount_cumulative_undiscounted',
 'expected_indirect_loss_amount_cumulative_discounted',
 'expected_indirect_loss_amount_cumulative_undiscounted',
 'expected_revenue_loss_amount_cumulative_discounted',
 'expected_revenue_loss_amount_cumulative_undiscounted',
 'expected_total_loss_amount_cumulative_discounted',
 'expected_total_loss_amount_cumulative_undiscounted',
 'indirect_loss_ratio_undiscounted',
 'total_loss_ratio_undiscounted'}

In [27]:
len(set(exposure_datapoints_df.columns).union(set(max_values.columns)) )# - set(exposure_datapoints_df)

43

In [28]:
max_values.columns

['entity_id',
 'rcp_scenario',
 'clientindustryid',
 'total_loss_ratio_discounted',
 'indirect_loss_ratio_discounted',
 'direct_loss_ratio_discounted',
 'max_total_loss_ratio_discounted',
 'max_indirect_loss_ratio_discounted',
 'max_direct_loss_ratio_discounted',
 'adjusted_total_loss_ratio_discounted',
 'adjusted_direct_loss_ratio_discounted',
 'adjusted_indirect_loss_ratio_discounted',
 'maintain_order',
 'overall_percentile_total_loss_ratio',
 'total_loss_ratio_rank_overall',
 'industry_percentile_total_loss_ratio',
 'total_loss_ratio_rank_industry',
 'overall_percentile_direct_loss_ratio',
 'industry_percentile_direct_loss_ratio',
 'overall_percentile_indirect_loss_ratio',
 'industry_percentile_indirect_loss_ratio',
 'percent_total_loss_from_direct',
 'percent_total_loss_from_indirect',
 'overall_category_direct_loss_ratio',
 'industry_category_direct_loss_ratio',
 'overall_category_indirect_loss_ratio',
 'industry_category_indirect_loss_ratio',
 'total_loss_ratio_category']

In [29]:
# final_exposure = exposure_datapoints_df.merge(max_values[['entity_id','rcp_scenario',
# 'adjusted_direct_loss_ratio_discounted',
#  'adjusted_indirect_loss_ratio_discounted',
#  'adjusted_total_loss_ratio_discounted',
#  'industry_category_direct_loss_ratio',
#  'industry_category_indirect_loss_ratio',
#  'industry_percentile_direct_loss_ratio',
#  'industry_percentile_indirect_loss_ratio',
#  'industry_percentile_total_loss_ratio',
#  'max_direct_loss_ratio_discounted',
#  'max_indirect_loss_ratio_discounted',
#  'max_total_loss_ratio_discounted',
#  'overall_category_direct_loss_ratio',
#  'overall_category_indirect_loss_ratio',
#  'overall_percentile_direct_loss_ratio',
#  'overall_percentile_indirect_loss_ratio',
#  'overall_percentile_total_loss_ratio',
#  'percent_total_loss_from_direct',
#  'percent_total_loss_from_indirect',
#  'total_loss_ratio_category',
#  'total_loss_ratio_rank_industry',
#  'total_loss_ratio_rank_overall']], on=['entity_id','rcp_scenario'], how = 'left')

In [30]:
final_exposure = exposure_datapoints_df.join(max_values[['entity_id','rcp_scenario',
'adjusted_direct_loss_ratio_discounted',
 'adjusted_indirect_loss_ratio_discounted',
 'adjusted_total_loss_ratio_discounted',
 'industry_category_direct_loss_ratio',
 'industry_category_indirect_loss_ratio',
 'industry_percentile_direct_loss_ratio',
 'industry_percentile_indirect_loss_ratio',
 'industry_percentile_total_loss_ratio',
 'max_direct_loss_ratio_discounted',
 'max_indirect_loss_ratio_discounted',
 'max_total_loss_ratio_discounted',
 'overall_category_direct_loss_ratio',
 'overall_category_indirect_loss_ratio',
 'overall_percentile_direct_loss_ratio',
 'overall_percentile_indirect_loss_ratio',
 'overall_percentile_total_loss_ratio',
 'percent_total_loss_from_direct',
 'percent_total_loss_from_indirect',
 'total_loss_ratio_category',
 'total_loss_ratio_rank_industry',
 'total_loss_ratio_rank_overall']], on=['entity_id','rcp_scenario'], how = 'left',coalesce=True).sort(by='entity_id')

final_exposure

entity_id,rcp_scenario,expected_indirect_loss_amount_cumulative_undiscounted,expected_indirect_loss_amount_cumulative_discounted,expected_revenue_loss_amount_cumulative_undiscounted,expected_revenue_loss_amount_cumulative_discounted,expected_asset_damage_loss_amount_cumulative_undiscounted,expected_asset_damage_loss_amount_cumulative_discounted,cash_flow_series_cumulative_discounted,cash_flow_series_cumulative_undiscounted,clientindustryid,expected_direct_damage_loss_amount_cumulative_undiscounted,expected_total_loss_amount_cumulative_undiscounted,indirect_loss_ratio_undiscounted,direct_loss_ratio_undiscounted,total_loss_ratio_undiscounted,expected_direct_damage_loss_amount_cumulative_discounted,expected_total_loss_amount_cumulative_discounted,indirect_loss_ratio_discounted,direct_loss_ratio_discounted,total_loss_ratio_discounted,adjusted_direct_loss_ratio_discounted,adjusted_indirect_loss_ratio_discounted,adjusted_total_loss_ratio_discounted,industry_category_direct_loss_ratio,industry_category_indirect_loss_ratio,industry_percentile_direct_loss_ratio,industry_percentile_indirect_loss_ratio,industry_percentile_total_loss_ratio,max_direct_loss_ratio_discounted,max_indirect_loss_ratio_discounted,max_total_loss_ratio_discounted,overall_category_direct_loss_ratio,overall_category_indirect_loss_ratio,overall_percentile_direct_loss_ratio,overall_percentile_indirect_loss_ratio,overall_percentile_total_loss_ratio,percent_total_loss_from_direct,percent_total_loss_from_indirect,total_loss_ratio_category,total_loss_ratio_rank_industry,total_loss_ratio_rank_overall
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,str,i32,i32
1007896773,"""RCP2.6""",2.8301e8,1.0364e8,5.6149e8,2.1143e8,5.1009e8,1.9410e8,2.7502e10,6.3446e10,201025,1.0716e9,1.3546e9,0.004461,0.01689,0.02135,4.0553e8,5.0917e8,0.003768,0.014746,0.018514,0.014746,0.003768,0.018514,"""Negligible""","""Negligible""",1.0,1.0,1.0,94.440072,6.132625,100.572698,"""High""","""High""",53.0,67.0,55.0,79.646154,20.353846,"""Negligible""",43,2653
1007896917,"""RCP2.6""",5.4395e7,1.9787e7,1.1731e8,4.3935e7,4.2215e8,1.5619e8,1.2878e10,2.9708e10,151015,5.3946e8,5.9386e8,0.001831,0.018159,0.01999,2.0012e8,2.1991e8,0.001537,0.01554,0.017077,0.01554,0.001537,0.017077,"""Low""","""Negligible""",2.0,1.0,2.0,94.440072,6.132625,100.572698,"""High""","""High""",55.0,38.0,52.0,91.002272,8.997728,"""Negligible""",111,2530
1007896995,"""RCP2.6""",3.3661e9,1.2071e9,9.4624e9,3.5411e9,1.0044e10,3.7844e9,6.9379e11,1.6006e12,255040,1.9507e10,2.2873e10,0.002103,0.012188,0.014291,7.3254e9,8.5325e9,0.00174,0.010559,0.012298,0.010559,0.00174,0.012298,"""Negligible""","""Negligible""",1.0,1.0,1.0,94.440072,6.132625,100.572698,"""High""","""High""",43.0,42.0,41.0,85.853216,14.146784,"""Negligible""",67,2004
1007897937,"""RCP2.6""",1.4758e8,5.4398e7,6.7413e8,2.5709e8,3.3269e8,1.2851e8,1.9160e11,4.4202e11,452051,1.0068e9,1.1544e9,0.000334,0.002278,0.002612,3.8559e8,4.3999e8,0.000284,0.002012,0.002296,0.002012,0.000284,0.002296,"""Negligible""","""Negligible""",1.0,1.0,1.0,94.440072,6.132625,100.572698,"""Medium""","""Medium""",10.0,6.0,8.0,87.636546,12.363454,"""Negligible""",30,372
1007899373,"""RCP2.6""",5.4840e8,1.9922e8,1.6631e9,6.3526e8,2.8064e9,1.1102e9,3.5418e11,8.1709e11,453052,4.4695e9,5.0179e9,0.000671,0.00547,0.006141,1.7454e9,1.9446e9,0.000562,0.004928,0.00549,0.004928,0.000562,0.00549,"""Low""","""Negligible""",2.0,1.0,2.0,94.440072,6.132625,100.572698,"""Medium""","""Medium""",23.0,14.0,20.0,89.755619,10.244381,"""Negligible""",79,950
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2013403858,"""RCP2.6""",1.9347e6,719120.837924,4.5460e6,1.7430e6,7809.807867,3001.62385,1.0936e9,2.5230e9,101012,4.5538e6,6.4884e6,0.000767,0.001805,0.002572,1.7460e6,2.4651e6,0.000658,0.001596,0.002254,0.001596,0.000658,0.002254,"""Negligible""","""Negligible""",0.0,1.0,0.0,94.44007

In [31]:
final_exposure = final_exposure.with_columns(
    pl.when(pl.col('indirect_loss_ratio_discounted') < 0)
        .then(pl.col('indirect_loss_ratio_discounted'))
        .otherwise(pl.col('indirect_loss_ratio_discounted').round(2))
        .alias('indirect_loss_ratio_discounted')
)

final_exposure = final_exposure.with_columns(
    pl.when(pl.col('direct_loss_ratio_discounted') < 0)
        .then(pl.col('direct_loss_ratio_discounted'))
        .otherwise(pl.col('direct_loss_ratio_discounted').round(2))
        .alias('direct_loss_ratio_discounted')
)

final_exposure = final_exposure.with_columns(
    (
        pl.when(pl.col('indirect_loss_ratio_discounted') < 0)
            .then(pl.col('indirect_loss_ratio_discounted'))
            .otherwise(pl.col('indirect_loss_ratio_discounted').round(2))
        + 
        pl.when(pl.col('direct_loss_ratio_discounted') < 0)
            .then(pl.col('direct_loss_ratio_discounted'))
            .otherwise(pl.col('direct_loss_ratio_discounted').round(2))
    ).alias('total_loss_ratio_discounted')
)

final_exposure = final_exposure.with_columns(
    (
        pl.col('expected_revenue_loss_amount_cumulative_discounted') / 
        pl.col('expected_total_loss_amount_cumulative_discounted') * 100
    ).alias('percent_of_total_loss_from_revenue_impacting_loss')
)

final_exposure = final_exposure.with_columns(
    (
        pl.col('expected_asset_damage_loss_amount_cumulative_discounted') / 
        pl.col('expected_total_loss_amount_cumulative_discounted') * 100
    ).alias('percent_of_total_loss_from_asset_damage_loss')
)

In [32]:
# final_exposure['indirect_loss_ratio_discounted'] = np.where(
#     final_exposure['indirect_loss_ratio_discounted'] < 0,
#     final_exposure['indirect_loss_ratio_discounted'],
#     final_exposure['indirect_loss_ratio_discounted'].round(2)
# )

# final_exposure['direct_loss_ratio_discounted'] = np.where(
#     final_exposure['direct_loss_ratio_discounted'] < 0,
#     final_exposure['direct_loss_ratio_discounted'],
#     final_exposure['direct_loss_ratio_discounted'].round(2)
# )

# final_exposure['total_loss_ratio_discounted'] = (
#     np.where(
#         final_exposure['indirect_loss_ratio_discounted'] < 0,
#         final_exposure['indirect_loss_ratio_discounted'],
#         final_exposure['indirect_loss_ratio_discounted'].round(2)
#     ) + np.where(
#         final_exposure['direct_loss_ratio_discounted'] < 0,
#         final_exposure['direct_loss_ratio_discounted'],
#         final_exposure['direct_loss_ratio_discounted'].round(2)
#     )
# )

# final_exposure['percent_of_total_loss_from_revenue_impacting_loss'] = (
#     final_exposure['expected_revenue_loss_amount_cumulative_discounted'] / 
#     final_exposure['expected_total_loss_amount_cumulative_discounted'] * 100
# )


# final_exposure['percent_of_total_loss_from_asset_damage_loss'] = (
#     final_exposure['expected_asset_damage_loss_amount_cumulative_discounted'] / 
#     final_exposure['expected_total_loss_amount_cumulative_discounted'] * 100
# )

In [33]:
final_exposure = final_exposure[["entity_id" , "rcp_scenario" , "expected_revenue_loss_amount_cumulative_undiscounted" , "expected_asset_damage_loss_amount_cumulative_undiscounted" , "expected_direct_damage_loss_amount_cumulative_undiscounted" , "expected_indirect_loss_amount_cumulative_undiscounted" , "indirect_loss_ratio_undiscounted" , "direct_loss_ratio_undiscounted" , "total_loss_ratio_undiscounted" , "expected_revenue_loss_amount_cumulative_discounted" , "expected_asset_damage_loss_amount_cumulative_discounted" , "expected_direct_damage_loss_amount_cumulative_discounted" , "expected_indirect_loss_amount_cumulative_discounted" , "indirect_loss_ratio_discounted" , "direct_loss_ratio_discounted" , "total_loss_ratio_discounted" , "overall_percentile_total_loss_ratio" , "industry_percentile_total_loss_ratio" , "overall_percentile_direct_loss_ratio" , "overall_category_direct_loss_ratio" , "industry_percentile_direct_loss_ratio" , "percent_total_loss_from_direct" , "overall_percentile_indirect_loss_ratio" , "overall_category_indirect_loss_ratio" , "industry_percentile_indirect_loss_ratio" , "percent_total_loss_from_indirect" , "total_loss_ratio_category" , "total_loss_ratio_rank_overall" , "total_loss_ratio_rank_industry" , "expected_total_loss_amount_cumulative_undiscounted" , "expected_total_loss_amount_cumulative_discounted" , "percent_of_total_loss_from_revenue_impacting_loss" , "percent_of_total_loss_from_asset_damage_loss"]]

In [37]:
final_exposure.write_csv("exposure_calculated_polars.csv")